# Model Comparison Framework
## Comparing Algorithms for FDI Stock Volatility Prediction

This notebook compares multiple forecasting algorithms across different complexity levels:

| Nhóm     | Thuật toán              | Vai trò           |
|----------|-------------------------|-------------------|
| Baseline | Historical Mean / ARIMA | Mốc so sánh        |
| ML       | Random Forest           | Phi tuyến          |
| DL       | LSTM                    | Quan hệ thời gian  |
| DL (opt) | GRU                     | So sánh nội bộ     |

**Goal**: Test each algorithm and recommend the optimal one based on performance metrics and research needs.

## Section 1: Import Required Libraries

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

# Import our model comparison framework
from model_comparison import (
    ModelComparator, 
    HistoricalMeanModel, 
    ARIMAModel,
    RandomForestVolatilityModel,
    LSTMVolatilityModel,
    GRUVolatilityModel,
    create_sequences,
    split_data
)

print("✓ All libraries imported successfully!")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

## Section 2: Load and Explore Dataset

In [ ]:
# Load volatility data from prepared data
# Assuming you've run 1_data_preparation.ipynb and generated volatility metrics

# For now, we'll use sample data. Replace with your actual data:
data_path = '../data/values.csv'  # Use actual data after preparation

if os.path.exists(data_path):
    data = pd.read_csv(data_path)
    print(f"✓ Data loaded: {data.shape}")
else:
    print("⚠ Data file not found. Using sample data.")
    # Generate sample volatility data for demonstration
    np.random.seed(42)
    dates = pd.date_range(start='2022-01-01', periods=250, freq='D')
    # Simulate stock price
    price = 100 + np.cumsum(np.random.randn(250) * 2)
    # Calculate rolling volatility (20-day window)
    returns = np.log(price[1:] / price[:-1])
    volatility = pd.Series(returns).rolling(window=20).std() * np.sqrt(252)
    
    data = pd.DataFrame({
        'date': dates,
        'volatility': volatility.values
    })
    
    print(f"✓ Sample data generated: {data.shape}")

print(f"\nData Info:")
print(f"  Shape: {data.shape}")
print(f"  Columns: {data.columns.tolist()}")
print(f"\nFirst few rows:")
print(data.head())
print(f"\nStatistics:")
print(data.describe())

### Visualize Time Series

In [ ]:
# Extract volatility values
if 'volatility' in data.columns:
    volatility_data = data['volatility'].dropna().values
else:
    # Use first numeric column
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    volatility_data = data[numeric_cols[0]].dropna().values

# Plot time series
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(volatility_data, linewidth=1.5)
plt.title('Stock Volatility Time Series', fontsize=12, fontweight='bold')
plt.xlabel('Time (days)')
plt.ylabel('Volatility')
plt.grid(True, alpha=0.3)

# Plot distribution
plt.subplot(1, 2, 2)
plt.hist(volatility_data, bins=30, edgecolor='black', alpha=0.7)
plt.title('Volatility Distribution', fontsize=12, fontweight='bold')
plt.xlabel('Volatility Value')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Data Statistics:")
print(f"  Mean: {np.mean(volatility_data):.6f}")
print(f"  Std:  {np.std(volatility_data):.6f}")
print(f"  Min:  {np.min(volatility_data):.6f}")
print(f"  Max:  {np.max(volatility_data):.6f}")

## Section 3: Data Preprocessing and Feature Engineering

In [ ]:
# Remove NaN values
volatility_clean = volatility_data[~np.isnan(volatility_data)]

print(f"Clean data shape: {volatility_clean.shape}")
print(f"Removed NaN values: {len(volatility_data) - len(volatility_clean)}")

# Normalize data to [0, 1] range
scaler = StandardScaler()
volatility_normalized = scaler.fit_transform(volatility_clean.reshape(-1, 1)).flatten()

print(f"\nNormalized data:")
print(f"  Mean: {np.mean(volatility_normalized):.6f}")
print(f"  Std:  {np.std(volatility_normalized):.6f}")

# Create lag features for ML models
def create_lag_features(data, lags=[1, 5, 10, 20]):
    """Create lag features for ML models"""
    df = pd.DataFrame()
    df['target'] = data
    
    for lag in lags:
        df[f'lag_{lag}'] = data.shift(lag)
    
    # Rolling statistics
    df['rolling_mean_5'] = pd.Series(data).rolling(5).mean()
    df['rolling_std_5'] = pd.Series(data).rolling(5).std()
    df['rolling_mean_10'] = pd.Series(data).rolling(10).mean()
    df['rolling_std_10'] = pd.Series(data).rolling(10).std()
    
    return df.dropna()

ml_features = create_lag_features(volatility_normalized)
print(f"\nML Features shape: {ml_features.shape}")
print(f"Features: {ml_features.columns.tolist()}")

## Section 4: Train-Test Split for Time Series

In [ ]:
# Split ratio: 60% train, 20% val, 20% test
lookback = 20

# For RNN models: create sequences
X_rnn, y_rnn = create_sequences(volatility_normalized, lookback)
(X_train_rnn, y_train_rnn), (X_val_rnn, y_val_rnn), (X_test_rnn, y_test_rnn) = split_data(X_rnn, y_rnn)

print(f"RNN Sequences:")
print(f"  X_train: {X_train_rnn.shape}")
print(f"  X_val:   {X_val_rnn.shape}")
print(f"  X_test:  {X_test_rnn.shape}")

# For ML models: use lag features
X_ml = ml_features.drop('target', axis=1).values
y_ml = ml_features['target'].values

n = len(X_ml)
train_idx = int(n * 0.6)
val_idx = int(n * 0.8)

X_train_ml = X_ml[:train_idx]
y_train_ml = y_ml[:train_idx]
X_test_ml = X_ml[val_idx:]
y_test_ml = y_ml[val_idx:]

print(f"\nML Features:")
print(f"  X_train: {X_train_ml.shape}")
print(f"  X_test:  {X_test_ml.shape}")

# For Baseline models: use full time series
train_baseline = volatility_normalized[:train_idx]
test_baseline = volatility_normalized[val_idx:]

print(f"\nBaseline Data:")
print(f"  Train: {len(train_baseline)}")
print(f"  Test:  {len(test_baseline)}")

## Section 5: Baseline Models - Historical Mean and ARIMA

In [ ]:
print("="*60)
print("BASELINE: Historical Mean")
print("="*60)

# Historical Mean Model
hm_model = HistoricalMeanModel(window=20)
hm_model.fit(train_baseline)
hm_pred = hm_model.predict(test_baseline)

# Denormalize predictions
hm_pred_denorm = scaler.inverse_transform(hm_pred.reshape(-1, 1)).flatten()
test_denorm = scaler.inverse_transform(test_baseline.reshape(-1, 1)).flatten()

# Calculate metrics
hm_rmse = np.sqrt(mean_squared_error(test_denorm, hm_pred_denorm))
hm_mae = mean_absolute_error(test_denorm, hm_pred_denorm)
hm_mape = mean_absolute_percentage_error(test_denorm, hm_pred_denorm)

print(f"\nHistorical Mean Results:")
print(f"  RMSE: {hm_rmse:.6f}")
print(f"  MAE:  {hm_mae:.6f}")
print(f"  MAPE: {hm_mape:.6f}")

In [ ]:
print("="*60)
print("BASELINE: ARIMA Model")
print("="*60)

try:
    from statsmodels.tsa.arima.model import ARIMA
    
    # Fit ARIMA model (1,1,1) - standard for financial data
    print("\nFitting ARIMA(1,1,1)...")
    arima_model = ARIMA(train_baseline, order=(1, 1, 1))
    arima_fit = arima_model.fit()
    
    # Forecast
    arima_forecast = arima_fit.get_forecast(steps=len(test_baseline))
    arima_pred = arima_forecast.predicted_mean.values
    
    # Denormalize
    arima_pred_denorm = scaler.inverse_transform(arima_pred.reshape(-1, 1)).flatten()
    
    # Calculate metrics
    arima_rmse = np.sqrt(mean_squared_error(test_denorm, arima_pred_denorm))
    arima_mae = mean_absolute_error(test_denorm, arima_pred_denorm)
    arima_mape = mean_absolute_percentage_error(test_denorm, arima_pred_denorm)
    
    print(f"\nARIMA(1,1,1) Results:")
    print(f"  RMSE: {arima_rmse:.6f}")
    print(f"  MAE:  {arima_mae:.6f}")
    print(f"  MAPE: {arima_mape:.6f}")
    
except ImportError:
    print("⚠ statsmodels not installed. Install with: pip install statsmodels")
    print("  Skipping ARIMA model.")
    arima_rmse = np.inf
    arima_mae = np.inf
    arima_mape = np.inf

## Section 6: Random Forest for Time Series Forecasting

In [ ]:
print("="*60)
print("ML MODEL: Random Forest")
print("="*60)

# Fit Random Forest
print("\nTraining Random Forest...")
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train_ml, y_train_ml)

# Predict
rf_pred = rf_model.predict(X_test_ml)

# Denormalize
rf_pred_denorm = scaler.inverse_transform(rf_pred.reshape(-1, 1)).flatten()
test_ml_denorm = scaler.inverse_transform(y_test_ml.reshape(-1, 1)).flatten()

# Calculate metrics
rf_rmse = np.sqrt(mean_squared_error(test_ml_denorm, rf_pred_denorm))
rf_mae = mean_absolute_error(test_ml_denorm, rf_pred_denorm)
rf_mape = mean_absolute_percentage_error(test_ml_denorm, rf_pred_denorm)

print(f"\nRandom Forest Results:")
print(f"  RMSE: {rf_rmse:.6f}")
print(f"  MAE:  {rf_mae:.6f}")
print(f"  MAPE: {rf_mape:.6f}")

# Feature importance
feature_names = ml_features.drop('target', axis=1).columns
importances = rf_model.feature_importances_
print(f"\nTop 5 Important Features:")
for i, (name, importance) in enumerate(sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)[:5]):
    print(f"  {i+1}. {name}: {importance:.4f}")

## Section 7: LSTM Model Implementation

In [ ]:
print("="*60)
print("DL MODEL: LSTM")
print("="*60)

try:
    # Reshape for LSTM (samples, timesteps, features)
    X_train_lstm = X_train_rnn.reshape((X_train_rnn.shape[0], X_train_rnn.shape[1], 1))
    X_test_lstm = X_test_rnn.reshape((X_test_rnn.shape[0], X_test_rnn.shape[1], 1))
    
    print(f"\nTraining LSTM...")
    lstm_model = LSTMVolatilityModel(lookback=lookback, lstm_units=50, epochs=50, batch_size=16)
    lstm_model.fit(X_train_lstm, y_train_rnn)
    
    # Predict
    lstm_pred = lstm_model.predict(X_test_lstm).flatten()
    
    # Denormalize
    lstm_pred_denorm = scaler.inverse_transform(lstm_pred.reshape(-1, 1)).flatten()
    y_test_lstm_denorm = scaler.inverse_transform(y_test_rnn.reshape(-1, 1)).flatten()
    
    # Calculate metrics
    lstm_rmse = np.sqrt(mean_squared_error(y_test_lstm_denorm, lstm_pred_denorm))
    lstm_mae = mean_absolute_error(y_test_lstm_denorm, lstm_pred_denorm)
    lstm_mape = mean_absolute_percentage_error(y_test_lstm_denorm, lstm_pred_denorm)
    
    print(f"\nLSTM Results:")
    print(f"  RMSE: {lstm_rmse:.6f}")
    print(f"  MAE:  {lstm_mae:.6f}")
    print(f"  MAPE: {lstm_mape:.6f}")
    
except Exception as e:
    print(f"⚠ LSTM training error: {e}")
    print("  Install TensorFlow: pip install tensorflow")
    lstm_rmse = np.inf
    lstm_mae = np.inf
    lstm_mape = np.inf

## Section 8: GRU Model Implementation

In [ ]:
print("="*60)
print("DL MODEL: GRU")
print("="*60)

try:
    # Reshape for GRU (samples, timesteps, features)
    X_train_gru = X_train_rnn.reshape((X_train_rnn.shape[0], X_train_rnn.shape[1], 1))
    X_test_gru = X_test_rnn.reshape((X_test_rnn.shape[0], X_test_rnn.shape[1], 1))
    
    print(f"\nTraining GRU...")
    gru_model = GRUVolatilityModel(lookback=lookback, gru_units=50, epochs=50, batch_size=16)
    gru_model.fit(X_train_gru, y_train_rnn)
    
    # Predict
    gru_pred = gru_model.predict(X_test_gru).flatten()
    
    # Denormalize
    gru_pred_denorm = scaler.inverse_transform(gru_pred.reshape(-1, 1)).flatten()
    y_test_gru_denorm = scaler.inverse_transform(y_test_rnn.reshape(-1, 1)).flatten()
    
    # Calculate metrics
    gru_rmse = np.sqrt(mean_squared_error(y_test_gru_denorm, gru_pred_denorm))
    gru_mae = mean_absolute_error(y_test_gru_denorm, gru_pred_denorm)
    gru_mape = mean_absolute_percentage_error(y_test_gru_denorm, gru_pred_denorm)
    
    print(f"\nGRU Results:")
    print(f"  RMSE: {gru_rmse:.6f}")
    print(f"  MAE:  {gru_mae:.6f}")
    print(f"  MAPE: {gru_mape:.6f}")
    
except Exception as e:
    print(f"⚠ GRU training error: {e}")
    print("  Install TensorFlow: pip install tensorflow")
    gru_rmse = np.inf
    gru_mae = np.inf
    gru_mape = np.inf

## Section 9: Model Evaluation and Comparison

In [ ]:
# Create comparison table
results_data = {
    'Model': ['Historical Mean', 'ARIMA(1,1,1)', 'Random Forest', 'LSTM', 'GRU'],
    'Group': ['Baseline', 'Baseline', 'ML', 'DL', 'DL'],
    'RMSE': [hm_rmse, arima_rmse, rf_rmse, lstm_rmse, gru_rmse],
    'MAE': [hm_mae, arima_mae, rf_mae, lstm_mae, gru_mae],
    'MAPE': [hm_mape, arima_mape, rf_mape, lstm_mape, gru_mape]
}

results_df = pd.DataFrame(results_data)
results_df = results_df.sort_values('RMSE').reset_index(drop=True)

print("\n" + "="*80)
print("MODEL COMPARISON TABLE")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

# Calculate improvement
best_rmse = results_df['RMSE'].min()
baseline_rmse = results_df[results_df['Group'] == 'Baseline']['RMSE'].min()
improvement = ((baseline_rmse - best_rmse) / baseline_rmse) * 100

print(f"\nImprovement over best baseline: {improvement:.2f}%")

### Detailed Comparison by Metric

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# RMSE Comparison
axes[0].barh(results_df['Model'], results_df['RMSE'], color=['red' if g == 'Baseline' else 'blue' if g == 'ML' else 'green' 
                                                                for g in results_df['Group']])
axes[0].set_xlabel('RMSE (Lower is Better)', fontweight='bold')
axes[0].set_title('Root Mean Squared Error', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# MAE Comparison
axes[1].barh(results_df['Model'], results_df['MAE'], color=['red' if g == 'Baseline' else 'blue' if g == 'ML' else 'green' 
                                                               for g in results_df['Group']])
axes[1].set_xlabel('MAE (Lower is Better)', fontweight='bold')
axes[1].set_title('Mean Absolute Error', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

# MAPE Comparison
axes[2].barh(results_df['Model'], results_df['MAPE'], color=['red' if g == 'Baseline' else 'blue' if g == 'ML' else 'green' 
                                                                for g in results_df['Group']])
axes[2].set_xlabel('MAPE % (Lower is Better)', fontweight='bold')
axes[2].set_title('Mean Absolute Percentage Error', fontweight='bold')
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("Legend:")
print("  Red   = Baseline Models")
print("  Blue  = ML Models")
print("  Green = DL Models")

## Section 10: Visualize Predictions

In [ ]:
# Align all predictions for visualization
# Note: Different models use different test sets, so we need to adjust

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Historical Mean
axes[0, 0].plot(test_denorm, 'o-', label='Actual', linewidth=2, markersize=4)
axes[0, 0].plot(hm_pred_denorm, 's--', label='Predicted', linewidth=2, markersize=4)
axes[0, 0].set_title(f'Historical Mean (RMSE: {hm_rmse:.4f})', fontweight='bold')
axes[0, 0].set_ylabel('Volatility')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# ARIMA
if arima_rmse != np.inf:
    axes[0, 1].plot(test_denorm, 'o-', label='Actual', linewidth=2, markersize=4)
    axes[0, 1].plot(arima_pred_denorm, 's--', label='Predicted', linewidth=2, markersize=4)
    axes[0, 1].set_title(f'ARIMA(1,1,1) (RMSE: {arima_rmse:.4f})', fontweight='bold')
    axes[0, 1].set_ylabel('Volatility')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# Random Forest
axes[0, 2].plot(test_ml_denorm, 'o-', label='Actual', linewidth=2, markersize=4)
axes[0, 2].plot(rf_pred_denorm, 's--', label='Predicted', linewidth=2, markersize=4)
axes[0, 2].set_title(f'Random Forest (RMSE: {rf_rmse:.4f})', fontweight='bold')
axes[0, 2].set_ylabel('Volatility')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# LSTM
if lstm_rmse != np.inf:
    axes[1, 0].plot(y_test_lstm_denorm, 'o-', label='Actual', linewidth=2, markersize=4)
    axes[1, 0].plot(lstm_pred_denorm, 's--', label='Predicted', linewidth=2, markersize=4)
    axes[1, 0].set_title(f'LSTM (RMSE: {lstm_rmse:.4f})', fontweight='bold')
    axes[1, 0].set_ylabel('Volatility')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# GRU
if gru_rmse != np.inf:
    axes[1, 1].plot(y_test_gru_denorm, 'o-', label='Actual', linewidth=2, markersize=4)
    axes[1, 1].plot(gru_pred_denorm, 's--', label='Predicted', linewidth=2, markersize=4)
    axes[1, 1].set_title(f'GRU (RMSE: {gru_rmse:.4f})', fontweight='bold')
    axes[1, 1].set_ylabel('Volatility')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

# Residuals
axes[1, 2].text(0.1, 0.9, "Model Comparison Summary\n", transform=axes[1, 2].transAxes, 
                fontsize=12, fontweight='bold', verticalalignment='top')
summary_text = "\n".join([f"{row['Model']}: RMSE={row['RMSE']:.4f}" 
                          for _, row in results_df.iterrows()])
axes[1, 2].text(0.1, 0.75, summary_text, transform=axes[1, 2].transAxes, 
                fontsize=10, verticalalignment='top', family='monospace')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## Section 11: Select Optimal Algorithm

In [ ]:
print("\n" + "="*80)
print("OPTIMAL ALGORITHM RECOMMENDATION")
print("="*80)

# Find best model by RMSE
best_model_idx = results_df['RMSE'].idxmin()
best_model = results_df.loc[best_model_idx]

print(f"\n✓ OPTIMAL ALGORITHM: {best_model['Model']} ({best_model['Group']})")
print(f"\nPerformance Metrics:")
print(f"  RMSE: {best_model['RMSE']:.6f}")
print(f"  MAE:  {best_model['MAE']:.6f}")
print(f"  MAPE: {best_model['MAPE']:.6f}%")

# Analysis
print(f"\n" + "="*80)
print("ANALYSIS & RECOMMENDATIONS")
print("="*80)

baseline_best = results_df[results_df['Group'] == 'Baseline'].iloc[0]
ml_best = results_df[results_df['Group'] == 'ML'].iloc[0]
dl_best = results_df[results_df['Group'] == 'DL'].iloc[0]

print(f"\n1. BASELINE (Reference Point):")
print(f"   Best: {baseline_best['Model']}")
print(f"   RMSE: {baseline_best['RMSE']:.6f}")
print(f"   Role: Provides benchmark for comparison")

print(f"\n2. MACHINE LEARNING:")
print(f"   Best: {ml_best['Model']}")
print(f"   RMSE: {ml_best['RMSE']:.6f}")
print(f"   Improvement: {((baseline_best['RMSE'] - ml_best['RMSE'])/baseline_best['RMSE']*100):.2f}% over baseline")
print(f"   Role: Captures non-linear patterns")

print(f"\n3. DEEP LEARNING:")
print(f"   Best: {dl_best['Model']}")
print(f"   RMSE: {dl_best['RMSE']:.6f}")
print(f"   Improvement: {((baseline_best['RMSE'] - dl_best['RMSE'])/baseline_best['RMSE']*100):.2f}% over baseline")
print(f"   Role: Captures temporal dependencies")

print(f"\n" + "="*80)
print("DECISION FRAMEWORK")
print("="*80)

print(f"\nChoose {best_model['Model']} because:")
print(f"  ✓ Lowest RMSE among all models")
print(f"  ✓ Group: {best_model['Group']} (balance between complexity and accuracy)")

if best_model['Group'] == 'Baseline':
    print(f"  ✓ Simple, interpretable, fast to train")
    print(f"  ✓ Low computational requirements")
elif best_model['Group'] == 'ML':
    print(f"  ✓ Captures non-linear relationships")
    print(f"  ✓ Feature importance available")
    print(f"  ✓ Fast training and inference")
else:
    print(f"  ✓ Captures long-term temporal dependencies")
    print(f"  ✓ Suitable for sequence modeling")
    print(f"  ✓ Can incorporate multiple features")

print(f"\n" + "="*80)
print("NEXT STEPS")
print("="*80)
print(f"\n1. Use {best_model['Model']} as primary model")
print(f"2. Fine-tune hyperparameters for better performance")
print(f"3. Validate on larger dataset (full 100 stocks)")
print(f"4. Compare with GNN-based approach (optional enhancement)")
print(f"5. Analyze FDI-specific patterns in predictions")